In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd 
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/refs/heads/master/adp/33/data/p1.csv")
df.head()

Category (종속변수)
- 0: 정상
- 1,2,3 (간염 정도, 심각성 : 3>2>1, 3은 간경화)

1-1 결측치 처리 방안 2개 제시하고, 처리 전후 비교 및 더 나은 방안 선택하여 전처리 완료

1-2 이상치를 확인하고 처리 여부 판단 후 작업

1-3 위 두 작업 적용 후 EDA 진행) 및 종속변수를 기준으로 각 독립변수의 차이가 있는지 확인하라 (시각화포함)

1-4 주어진 데이터로부터 주성분분석을 수행해 새로운 설명변수를 도출할 수 있는지를 검토 ∙ 판단한 뒤, 그 결과를 제시하시오

2-1 데이터 불균형이 왜 문제인지 설명하고 불균형 해결방법에 대해 두가지를 설명하라

2-2 불균형 데이터 분류모델에 적합한 평가지표 3개 제시하고 그 이유 설명하라

2-3 종속 변수를 정상(Category 0)과 비정상(Category 1,2,3)로 변경한 후 위의 2가지 불균형 처리 방법을 사용하고, 3가지 평가지표로 결과를 각각 평가하라.

(학습 규칙)

- 종속 변수를 이진화 후 불균형 처리 방법 진행
- 전처리 후 데이터 행의 index number를 초기화 하라(0부터 시작)
- index number 값이 5로 나눈 나머지가 0이라면 test, 그 외의 경우 train set으로 하여 진행
- 3가지 분류 모델을 사용하라

3-1 1에서 전처리 완료한 데이터를 바탕으로 종속변수에서 0을 제외한 1,2,3의 데이터만 필터하여 분류모델 학습 및 평가를 진행하라.

(학습 규칙)
- 전처리 후 데이터 행의 index number를 초기화 하라(0부터 시작)
- index number 값이 5로 나눈 나머지가 0이라면 test, 그 외의 경우 train set으로 하여 진행
- 3개의 분류 모델을 사용하라

3-2 학습한 모델 결과와 평가지표를 바탕으로 간염여부에 영향을 미치는 요소들에 대해 논의하라

데이터 설명
데이터 출처 : https://www.data.go.kr/data/15048032/fileData.do / 후처리, 통계청 날씨정보

데이터 링크 : url = https://raw.githubusercontent.com/Datamanim/datarepo/refs/heads/master/adp/33/data

url/p2_sb_2021.csv

url/p2_sb_2022.csv

url/p2_sb_2023.csv

url/p2_w_2021.csv

url/p2_w_2022.csv

url/p2_w_2023.csv

데이터 설명 :

21년~23년의 J시의 2개 지하철호선 지하철의 시간대별 승하차 인원 정보 데이터 (p2_sb_년도.csv)

X5~ X22 : 5시부터 22시 승차 또는 하차 인원, (X5 : 05시의 계절 날짜와 대응)

역번호 : 각 정류장의 고유 번호

승하차구분 : 승차 또는 하차

21년~23년의 J시의 특정지역구의 시간대별 게절성 정보 데이터 (p2_w_년도.csv)

시간별 J시의 날씨정보

In [ ]:
p = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/refs/heads/master/adp/33/data/p2_sb_2021.csv')
weather = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/refs/heads/master/adp/33/data/p2_w_2021.csv')
display(p.head())
display(weather.head())

4-1 아래 조건들을 참고하여 전처리를 시행하고 훈련데이터에 대한 기초통계량을 구하시오

승하차 데이터에서 X5~X22 모든컬럼이 0의 값을 가지는 경우 해당 날짜,호선,역번호 승하차 행은 제외한다.

승하차 데이터를 날짜,호선,역번호,시간대별(05~22시) 승객수의 총합을 구하여라(승차 + 하차), 결측치는 0명으로 간주한다. 컬럼명은 users로 변경한다.

날짜 데이터의 강수 데이터는 결측치를 0, 그 외 컬럼의 결측치는 직전값으로 대치한다.

날짜데이터와 일자,시간별 승하차인원 데이터를 동일날짜,시간에 대해 합친다.

일자정보를 바탕으로 주말 여부를 나타내는 weekday컬럼을 추가한다. (주말=1,평일=0)

21년,22년 데이터는 훈련데이터, 23년 데이터는 테스트 데이터로 사용한다.

4-2 전체 년도 데이터에 대해 users와 날씨 정보에 대해 상관관계를 분석하고 주말여부에 따른 users 변수의 차이가 존재하는지 통계적으로 확인하라.

4-3 users를 예측하는 2가지 회귀모델을 학습하고 모델에 대해 평가하라 (선형회귀 모델은 1개 필수 포함)
통계 (40점)
데이터 설명
데이터 출처 : 자체 제작

데이터 링크 : ./data/s1.csv

데이터 설명 : A,B 두 공장의 특정 일자의 수율(%)데이터. 단, 불량률은 (100 - 수율)로 정의한다.


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/refs/heads/master/adp/33/data/s1.csv')
df.head()

5-1 공장별 결측치, 기초통계량 등을 확인하라

5-2 A,B 공장의 불량률 차이가 존재하는지 확인하려 한다. 귀무,대립 가설을 설정하고 설명하라

5-3 A,B의 차이가 있는지 통계검정 하고 설명하라

데이터 설명
데이터 출처 : https://www.data.go.kr/data/15123586/fileData.do 후처리

데이터 링크 : ./data/s2.csv

데이터 설명 : 일자별 지하철 노션별 지연 데이터

In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/refs/heads/master/adp/33/data/s2.csv')
df.head()

6-1 노선에 상관없이 일별 최대 지연시간이 5~15분으로 발생하는 경우는 하나의 사건으로 보자. 해당 사건이 일자별 발생하는 빈도가 푸아송분포를 따르는지 확인하는 방법 2가지를 기술하고 결과를 보여라

6-2 10분이하, 10분초과 -20분 이하, 20분초과 3가지로 지연시간을 지연정도로 분류하라. 호선별 지연정도를 교차표로 구하여라

6-3 지하철 호선별로 지연정도가 차이가 있는지 확인하려한다. 귀무가설과 대립가설을 설정한 후 통계 검정을 실시하라

데이터 설명
데이터 출처 : 자체제작

데이터 링크 : ./data/s3.csv

데이터 설명 : 시계열 정보를 가지는 250행의 데이터



In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/refs/heads/master/adp/33/data/s3.csv')
df.head()

7-1 ar ma arma에 대해 설명하고, ACF, PACF는 패턴을 통해 설명하라.

7-2 Acf pacf를 그리고 해석하라, 해당 Plot data를 기반으로 ARMA(p,q)모형 제시하라